## Install Required Packages

In [ ]:
# %pip install --upgrade raiwidgets
# %pip install --upgrade pandas
# %pip install --upgrade fairlearn
# %pip install --upgrade interpret-community

After installing packages, you must close and reopen the notebook as well as restarting the kernel.

## Fairness Assessment

**Fairness Assessment** powered by [Fairlearn](https://github.com/fairlearn/fairlearn), which identifies which groups of people may be disproportionately negatively impacted by an AI system and in what ways.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from raiwidgets import FairnessDashboard
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression


In [ ]:
def load_custom_dataset():
    file_path = "/home/josh/Downloads/AiDashOwnModels/Acidents_dataset_v7.csv"
    dataset = pd.read_csv(file_path)
    return dataset

# Call the function to load dataset
df = load_custom_dataset()

# Check for missing values
if df.isnull().sum().any():
    print("Missing values detected in the dataset.")
else:
    print("No missing values in the dataset.")

# Display first few rows to verify
print(df.head())

# Select target column (label) and features
X_Raw = df
y_true = (df['Degree of Injury'] == 'Fatal').astype(int)

In [ ]:
sensitive_features = X_Raw[['Nature of Injury', 'Part of Body']]
X = X_Raw.drop(labels=['Nature of Injury', 'Part of Body'], axis=1)

# One-hot encoding for categorical variables
X = pd.get_dummies(X)

# Scale the features
sc = StandardScaler()
X_scaled = sc.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

# Encode the target variable
le = LabelEncoder()
y_true = le.fit_transform(y_true)

In [ ]:
X_train, X_test, y_train, y_test, sensitive_features_train, sensitive_features_test = \
    train_test_split(X_scaled, y_true, sensitive_features,
                     test_size=0.2, random_state=0, stratify=y_true)

# Work around indexing bug
X_train = X_train.reset_index(drop=True)
sensitive_features_train = sensitive_features_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
sensitive_features_test = sensitive_features_test.reset_index(drop=True)

In [ ]:
# Train the model
unmitigated_predictor = LogisticRegression(solver='liblinear', fit_intercept=True)
unmitigated_predictor.fit(X_train, y_train)

In [ ]:
# Make predictions
y_pred = unmitigated_predictor.predict(X_test)

# Check predictions
print("Unique values in y_test:", set(y_test))
print("Unique values in y_pred:", set(y_pred))
# Launch the Fairness Dashboard
FairnessDashboard(sensitive_features=sensitive_features_test,
                  y_true=y_test,
                  y_pred=y_pred,
                 )

## © Copyright, 2025 Assentian Limited. All Rights Reserved 